# Projet OList

## Préparation et imports

In [ ]:
import time

start_time = time.time()

In [ ]:
import pandas as pd, numpy as np, os, warnings, seaborn as sns, scipy.stats as st
from datetime import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
font1 = fm.FontProperties(size=20)
font2 = fm.FontProperties(size=24)

# %matplotlib inline
# %pylab inline

if int(str(sns.__version__).split('.')[1]) > 8 : 
    plt.style.use('seaborn-v0_8-darkgrid')
else:
    plt.style.use('seaborn-darkgrid')
    
sns.set(font_scale=2)
warnings.filterwarnings(action="ignore")

In [ ]:
palette_large = [ "#030aa7", "#e50000", "#d8863b", "#005f6a", "#6b7c85", "#751973", 
            "#0485d1", "#ff7855", "#fbeeac", "#0cb577", "#95a3a6", "#c071fe", 
            "#d1e5f0", "#fddbc7", "#ffffcb", "#12e193", "#d8dcd6", "#dfc5fe", 
          ]
palette = ['#b2182b','#008827','#053061','#ffa500', '#34495e', '#e74c3c', 
          '#9b59b6', '#0165fc','#2ecc71', '#cb7723', '#89a0b0']
sns.palplot(sns.color_palette(palette))
sns.palplot(sns.color_palette(palette_large))
plt.show()

In [ ]:
def formatPct(pct, allvals):
    total = int(round(pct/100. * np.sum(allvals)))
    return "{:.2f}%\n({:d})".format(pct, total) 

In [ ]:
donneesVente = pd.read_parquet('ecommerce/analyse_ventes.parquet', engine='pyarrow')
donneesAchat = pd.read_parquet('ecommerce/analyse_achat.parquet', engine='pyarrow')

#### Petite correction pour le parquet vente

In [ ]:
donneesVente["score"] = donneesVente.score_1+\
                          donneesVente.score_2*2+\
                          donneesVente.score_3*3+\
                          donneesVente.score_4*4+\
                          donneesVente.score_5*5

In [ ]:
donneesVente.set_index('order_id')

In [ ]:
donneesAchat.set_index('order_id')

In [ ]:
donneesAchat.columns

In [ ]:
donneesVente.columns

## Constitution des modèles

### Modèle Vente

#### Métriques

Les métriques sont:

- Prix: Prix du produit
    - value_ : Valeur  avec la méthode de payement
    - int_ : Nombre d'objet payé avec la méthode
- Satisfaction: Satisfaction du client sur le produit
    - score_: Note du produit
- Position des clients: Coordonnées géographiques
    - lat: Latitude
    - lng: Longitude
- Nombre de commande
    - order_id
- Temps de livraison
    - estimated_delivery - purchase_timestamp
    - delivered_customer - purchase_timestamp

#### Dimensions

Les dimensions sont: 

- Temporelle
    - Date: ``annee``, ``trimestre``, ``mois``, ``semaine``, ``jour``, ``annee_mois``, ``annee_jour``, ``jour_semaine``, ``annee_trimestre``, ``annee_semaine``
    - Suivi: ``purchase_timestamp``, ``approved_at``, ``delivered_carrier``, ``delivered_customer``, ``estimated_delivery``
- Type de payement: ``int_boleto``, ``int_credit_card``, ``int_debit__card``, ``int_not_defined``, ``int_voucher``
- Géographiques: ``zip_code``, ``city``, ``state``, ``state_name``, ``lat_min``, ``lat_max``, ``lat``, ``lng_min``, ``lng_max``, ``lng``
- Statut: ``status``, ``approuvee``, ``envoyee``, ``livree``, ``estimee``
- Satisfaction: ``score_1``, ``score_2``, ``score_3``, ``score_4``, ``score_5``, ``score``

#### Hiérarchies

Les hiérarchies sont:

- Temporelle:
    - Entonnoir: ``annee``, ``mois``, ``semaine``, ``jour``, ``heure``
    - Trimestrielle: ``anne``, ``trimestre``

- Geographique: ``city``, ``state``, ``zip_code``, ``state_name``, ``lat``, ``lng``

- Temporelle 2: ``purchase_timestamp``, ``approved_at``, ``delivered_carrier``, ``delivered_customer``, ``estimated_delivery``

### Modèle Achat

#### Métriques

Les métriques sont:
- Prix et coûts
  - ```price``` : prix de l'article
  - ```freight_value``` : coût de livraison
  - ```limit``` : valeur limite associée
- Caractéristiques des produits
  - ```name_lenght``` : longueur du nom du produit
  - ```description_lenght``` : longueur de la description
  - ```photos_qty``` : nombre de photos associées
- Dimensions physiques
  - ```weight_g``` : poids du produit en grammes
  - ```length_cm``` : longueur du produit en cm
  - ```height_cm``` : hauteur du produit en cm
  - ```width_cm``` : largeur du produit en cm
- Performances et délais
  - ```approuvee```, ```envoyee```, ```livree```, ```estimee``` : délais ou performances liés aux étapes de commande
- Coordonnées géographiques
  - ```cust_lat```, ```cust_lng```, ```sell_lat```, ```sell_lng``` : coordonnées géographiques des clients et vendeurs

#### Dimensions

Les dimensions sont:
- Identifiants
  - ```order_id```, ```order_item_id```, ```product_id```, ```seller_id```, ```customer_id```, ```customer_unique_id``` : identifiants uniques
- Catégorisation
  - ```category_name``` : catégorie du produit
  - ```status``` : état de la commande
- Localisation
  - Client : ```cust_zip_code```, ```cust_city```, ```cust_state```, ```cust_name_state```
  - Vendeur : ```sell_zip_code```, ```sell_city```, ```sell_state```, ```sell_name_state```
- Date et heure
  - ```shipping_limit```, ```purchase_timestamp```, ```approved_at```, ```delivered_carrier```, ```delivered_customer```, ```estimated_delivery```
- Temporelle:
    - ``annee``, ``mois``, ``anne_mois``, ``jour``, ``annee_jour``, ``jour_semaine``, ``trimestre``, ``annee_trimestre``, ``semaine``, ``annee_semaine``, ``heure``

#### Hiérarchies

Les hiérarchies sont:
- Géographique
  - Client : ```cust_zip_code```, ```cust_city```, ```cust_state```,  ```cust_name_state```
  - Vendeur : ```sell_zip_code```, ```sell_city```, ```sell_state```,  ```sell_name_state```
- Temporelle:
    - Entonnoir: ``annee``, ``mois``, ``semaine``, ``jour``, ``heure``
    - Trimestrielle: ``anne``, ``trimestre``


# Problématiques

## Ventes

Nous allons regarder l'évolution de la satisfaction globale.
Cette problèmatique est intéressante, car celle-ci permet d'avoir une meilleure vue de comment les clients aiment nos services

Afin de faire cette analyse, nous allons utiliser 3 dimensions:

    1. Temporelle
    2. Géographique
    3. Satisfaction

La raison du choix de la dimension temporelle, est de voir comment ce produit a évolué dans le temps, si les ventes ont augmentés ou diminués.
La raison de la dimension géographique est que certains produits peuvent avoir un attrait plus important dans certaines régions ou villes.
Et enfin, le score va permettre de savoir si un produit a été apprécié ou non.

\
\
\
Une autre analyse qui sera effectué sera celle de l'évolution des ventes à travers le temps.
Cette analyse est intéressante, car celle-ci permets de se faire une idée plus générale de l'implémentation de l'entreprise dans le pays, savoir dans quelle zone elle est plus active que d'autre.
Savoir quelles sont les moments les plus actifs, et quels sont les moyens de payements les plus utilisés

Afin de répondre à cette analyse, nous allons utiliser les 3 dimensions suivantes:

    1. Temporelle
    2. Géographique
    3. Type de payement

\
\
\
Une dernière analyse que nous allons effectuer sera celle d'une analyse de l'évolution des notes en fonction du temps et de la région.

Pour effectuer cette analyse, nous allons utiliser les 3 dimensions suivantes:

    1. Temporelle
    2. Géographique
    3. Satisfaction

## Achats

Notre première analyse portera sur l'évolution du rendement à travers le pays, afin de savoir où l'on perds de l'argent, et où l'on en gagne.

Les dimensions utilisés pour l'analyse: 

    1. Temporelle
    2. Géographique
    3. Catégorie

\
\
\
Notre deuxième analyse portera sur la localisation des vendeurs et clients

Les dimensions utilisés pour l'analyse: 

    1. Temporelle
    2. Géographique
    3. Catégorie

\
\
\
Notre dernière analyse sera sur l'évolution du comportement des clients

Les dimensions utilisés pour l'analyse: 

    1. Temporelle
    2. Géographique

In [ ]:
num_graph = 1

# Graphs

## Ventes

In [ ]:
dPayementBoleto = donneesVente['int_boleto'] * donneesVente['value_boleto']
dPayementCredit = donneesVente['int_credit_card'] * donneesVente['value_credit_card']
dPayementDebit = donneesVente['int_debit_card'] * donneesVente['value_debit_card']
dPayementVoucher = donneesVente['int_voucher'] * donneesVente['value_voucher']

donneesVente['Vente'] = dPayementBoleto + dPayementCredit + dPayementDebit + dPayementVoucher

### Analyse 1

In [ ]:
plt.figure(figsize=(36,12))

monthly_satisfaction = donneesVente.groupby('annee')['score'].sum().reset_index()

annee_unique = monthly_satisfaction['annee'].unique()
counter = 0

for month in annee_unique:
    subdf = donneesVente[donneesVente['annee'] == month]
    sns.histplot(subdf['mois'], kde=True, label=str(month), bins=20, color=palette[counter])
    counter+=1

del subdf
del monthly_satisfaction
del annee_unique

plt.xlabel("Mois", fontsize=12)
plt.ylabel("Nombre de satisfaction", fontsize=12)

plt.title(f"Figure {num_graph}: Evolution de la satifactions par mois")
plt.legend()
num_graph+=1

In [ ]:
radius,size=0.8,0.3
fig,ax = plt.subplots(ncols=1,figsize=(24,24), subplot_kw=dict(aspect="equal"))

affichage = donneesVente.groupby('state').city.count().reset_index().rename(columns={'city':'nombre'})
affichage['%'] = affichage.nombre * 100 / affichage.nombre.sum()

wedges, texts, autotexts =  ax.pie(
         affichage['nombre'],
         autopct=lambda pct: formatPct(pct, affichage.nombre.values),
         labels=affichage.state,
         counterclock=False,
         startangle=0 ,
         colors = palette,
         labeldistance=1.1, 
         textprops=dict(color="#030aa7")
      );

plt.title(f"Figure {num_graph}: Répartition des origines des scores")
plt.setp(autotexts, size=24, weight="bold",color="w")
plt.setp(texts, size=32, weight="bold");
num_graph+=1

In [ ]:
plt.figure(figsize=(36,12))
donneesVenteState = donneesVente.groupby("state").agg({
    "score" : "mean"
})

sns.barplot(
    donneesVenteState,
    x="state", 
    y="score",
    hue = "state"
)
plt.xlabel("Région", fontsize=12)
plt.ylabel("Moyenne de satisfaction globale", fontsize=12)

plt.title(f"Figure {num_graph}: Moyenne de la satisfaction par état")
plt.legend()
num_graph+=1

In [ ]:
satisfaction_by_state_time = donneesVente.groupby(['annee', 'state'])['score'].mean().reset_index()

plt.figure(figsize=(14, 8))
sns.lineplot(data=satisfaction_by_state_time, x='annee', y='score', hue='state', marker='o', palette='tab10')

plt.title(f"Figure {num_graph}: Satisfaction des clients par région par ans", fontsize=16)
plt.xlabel("Année", fontsize=12)
plt.ylabel("Moyenne du score de satisfaction", fontsize=12)
plt.legend(title="State", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)

num_graph+=1

In [ ]:
satisfaction_by_state_time = donneesVente.groupby(['annee_mois', 'state'])['score'].mean().reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(data=satisfaction_by_state_time, x='annee_mois', y='score', hue='state', marker='o', palette='tab10')


plt.title(f"Figure {num_graph}: Satisfaction des clients à travers les mois", fontsize=16)
plt.xlabel("Année-Mois", fontsize=12)
plt.ylabel("Moyenne du score de satisfaction", fontsize=12)
plt.legend(title="State", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
num_graph+=1

In [ ]:
satisfaction_by_state_time_month = donneesVente.groupby(['mois', 'state'])['score'].mean().reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(data=satisfaction_by_state_time_month, x='mois', y='score', hue='state', marker='o', palette='tab10')


plt.title(f"Figure {num_graph}: Satisfaction des clients à travers les mois", fontsize=16)
plt.xlabel("Mois", fontsize=12)
plt.ylabel("Moyenne du score de satisfaction", fontsize=12)
plt.legend(title="State", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
num_graph+=1
del satisfaction_by_state_time_month

In [ ]:
satisfaction_by_state_time_semaine = donneesVente.groupby(['semaine', 'state'])['score'].mean().reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(data=satisfaction_by_state_time_semaine, x='semaine', y='score', hue='state', marker='o', palette='tab10')


plt.title(f"Figure {num_graph}: Satisfaction des clients à travers les semaines", fontsize=16)
plt.xlabel("Semaine", fontsize=12)
plt.ylabel("Moyenne du score de satisfaction", fontsize=12)
plt.legend(title="State", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
num_graph+=1
del satisfaction_by_state_time_semaine

In [ ]:
satisfaction_over_time = donneesVente.groupby('annee_mois')['score'].mean().reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(data=satisfaction_over_time, x='annee_mois', y='score', marker='o', color='green', label='Average Score')


plt.title(f"Figure {num_graph}: Satisfaction globale des clients à travers les années", fontsize=16)
plt.xlabel("Année-mois", fontsize=12)
plt.ylabel("Moyenne du score de satisfaction", fontsize=12)
plt.grid(True)
num_graph+=1

### Analyse 2

In [ ]:
plt.figure(figsize=(36,12))

monthly_sales = donneesVente.groupby('annee')['Vente'].sum().reset_index()

annee_unique = monthly_sales['annee'].unique()
counter = 0

for month in annee_unique:
    subdf = donneesVente[donneesVente['annee'] == month]
    sns.histplot(subdf['mois'], kde=True, label=str(month), bins=20, color=palette[counter])
    counter+=1

plt.legend();
plt.title(f"Figure {num_graph}: Evolution des ventes à travers le temps")
num_graph+=1

In [ ]:
graph = sns.catplot(x="state", 
                    y="Vente", 
                    hue=donneesVente['annee'].astype(str),
                    data=donneesVente,
                    height=12, 
                    kind="bar", 
                    aspect=2,
                    palette=palette,
                    legend=False);

graph.despine(left=True);
graph.set_xticklabels(rotation=90);
graph.set_xlabels('');
graph.set_ylabels('');
plt.legend(loc='upper right', title_fontsize=20, fontsize=20);
plt.title(f"Figure {num_graph}: Vente à travers le temps par région")
num_graph+=1

In [ ]:
sales_by_state_year = donneesVente.groupby(['state', 'annee'])['Vente'].sum().reset_index()

heatmap_data = sales_by_state_year.pivot(index='state', columns='annee', values='Vente')

plt.figure(figsize=(14, 10))
sns.set(font_scale=1.1)
sns.heatmap(heatmap_data, cmap="YlGnBu", annot=True, fmt=".0f", linewidths=0.5, cbar_kws={'label': 'Total de vente (En Real)'})

plt.title(f"Figure {num_graph}: Total de vente par région par année", fontsize=16)
plt.xlabel("Année", fontsize=12)
plt.ylabel("Région", fontsize=12)
num_graph+=1


In [ ]:
sales_by_trimester_method = donneesVente.groupby(['trimestre', 'state'])['Vente'].sum().unstack(fill_value=0)

sales_by_trimester_method.plot(kind='bar', stacked=True, figsize=(12, 8), colormap='tab10')


plt.title(f"Figure {num_graph}:  Vente par état par trimestre", fontsize=16)
plt.xlabel("Trimeste", fontsize=12)
plt.ylabel("Total Sales (Vente)", fontsize=12)
plt.legend(title="State", bbox_to_anchor=(1.05, 1), loc='upper left')
num_graph+=1

In [ ]:
payment_columns = ['int_boleto', 'int_credit_card', 'int_debit_card', 'int_voucher', 'int_not_defined']

plt.figure(figsize=(12, 8))
for column in payment_columns:
    sns.histplot(
        data=donneesVente,
        x='annee',
        weights=column,
        kde=True,
        label=column,
        multiple='stack'
    )

plt.title(f"Figure {num_graph}: Evolution des moyens de payments à travers les années")
plt.xlabel('Année')
plt.ylabel('Number of d\'articles payés')
plt.legend(title='Méthode de payement')
plt.xticks(rotation=45)
num_graph+=1

In [ ]:
value_columns = ['value_boleto', 'value_credit_card', 'value_debit_card', 'value_voucher', 'value_not_defined']

payment_methods = [col.replace('value_', '').replace('_', ' ').capitalize() for col in value_columns]


plt.figure(figsize=(12, 8))

for i, column in enumerate(value_columns):
    
    x_values = [i] * len(donneesVente)
    plt.scatter(
        x_values, 
        donneesVente[column], 
        label=payment_methods[i], 
        alpha=0.7
    )


plt.title(f"Figure {num_graph}: Correlation entre les méthodes de payemnet et la quantité d\'argent", fontsize=16)
plt.xlabel('Méthode de payement', fontsize=14)
plt.ylabel('Quantité d\'argent', fontsize=14)
plt.xticks(range(len(value_columns)), payment_methods, rotation=45)
plt.legend(title='Méthode de payement', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
num_graph+=1

In [ ]:
usage_columns = ['value_boleto', 'value_credit_card', 'value_debit_card', 'value_voucher', 'value_not_defined']

usage_by_state = donneesVente.groupby('state')[usage_columns].sum()

plt.figure(figsize=(14, 8))
usage_by_state.plot(kind='bar', stacked=True, alpha=0.85, figsize=(14, 8), colormap='viridis')

plt.title(f"Figure {num_graph}: Correlation entre les moyens de payements et les régions", fontsize=16)
plt.xlabel('Régions', fontsize=14)
plt.ylabel('Nombre d\'achat', fontsize=14)
plt.legend(title='Méthode de payement', fontsize=12)
num_graph+=1

In [ ]:
amount_by_state = donneesVente.groupby('state')[usage_columns].sum()

plt.figure(figsize=(14, 8))
bar_width = 0.15
x = range(len(amount_by_state))

for i, column in enumerate(usage_columns):
    plt.bar(
        [pos + i * bar_width for pos in x], 
        amount_by_state[column], 
        width=bar_width, 
        label=column.replace('value_', '').capitalize()
    )

plt.title(f"Figure {num_graph}: Correlation entre l\'argent dépensé par chaque méthode de payement et les régions", fontsize=16)
plt.xlabel('Régions', fontsize=14)
plt.ylabel('Quantité d\'argent dépensée', fontsize=14)
plt.xticks([pos + bar_width * 2 for pos in x], amount_by_state.index, rotation=45)
plt.legend(title='Méthode de payement', fontsize=12)
plt.tight_layout()
num_graph+=1

In [ ]:
average_usage_by_state = donneesVente.groupby('state')[usage_columns].mean()

plt.figure(figsize=(14, 8))
sns.heatmap(average_usage_by_state, annot=True, fmt=".1f", cmap='viridis', cbar_kws={'label': 'Average Usage'})

plt.title(f"Figure {num_graph}: Moyenne d\'utilisation de chaque moyen de payment par région", fontsize=16)
plt.xlabel('Méthode de payment', fontsize=14)
plt.ylabel('Région', fontsize=14)
plt.tight_layout()
num_graph+=1

### Analyse 3

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(data=donneesVente, x='annee_mois', y='score', hue='state')
plt.xticks(rotation=45)
plt.title(f"Figure {num_graph}: Distribution des scores par mois et par région")
plt.xlabel("Date (année-mois)")
plt.ylabel("Score")
plt.legend(title='Région')
plt.tight_layout()
plt.show()
num_graph+=1

In [ ]:
donnees_2017 = donneesVente[donneesVente['purchase_timestamp'].dt.year >= 2017].copy()

donnees_2017['annee_mois'] = donnees_2017['purchase_timestamp'].dt.to_period('M').astype(str)

comment_cols = ['comment_1', 'comment_2', 'comment_3', 'comment_4', 'comment_5']

donnees_2017['avg_comment_length'] = donnees_2017[comment_cols].mean(axis=1)

df_evolution = donnees_2017.groupby(['annee_mois', 'state']).agg(
    score_moyen=('score', 'mean'),
    comment_length_moyen=('avg_comment_length', 'mean')
).reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(data=df_evolution, x='annee_mois', y='comment_length_moyen', hue='state', marker='o')
plt.xticks(rotation=45)
plt.title(f"Figure {num_graph}: Évolution de la longueur moyenne des commentaires (depuis 2017) par mois et par région")
plt.xlabel("Date (année-mois)")
plt.ylabel("Longueur moyenne des commentaires")
plt.legend(title='Région')
plt.tight_layout()
plt.show()
num_graph+=1

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_evolution, x='comment_length_moyen', y='score_moyen', hue='state', s=100)
plt.title(f"Figure {num_graph}: Relation entre score moyen et longueur moyenne des commentaires (depuis 2017)")
plt.xlabel("Longueur moyenne des commentaires")
plt.ylabel("Score moyen")
plt.legend(title='Région', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()
num_graph+=1

In [ ]:
donneesVente['purchase_timestamp'] = pd.to_datetime(donneesVente['purchase_timestamp'])

donnees_2017 = donneesVente[donneesVente['purchase_timestamp'].dt.year >= 2017].copy()

donnees_2017['annee_mois'] = donnees_2017['purchase_timestamp'].dt.to_period('M').astype(str)

df_scores = donnees_2017.groupby(['annee_mois', 'state'])['score'].mean().reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(data=df_scores, x='annee_mois', y='score', hue='state', marker='o')
plt.xticks(rotation=45)
plt.title(f"Figure {num_graph}: Évolution des scores moyens par mois et par région (depuis 2017)")
plt.xlabel("Date (année-mois)")
plt.ylabel("Score moyen")
plt.legend(title='Région')
num_graph+=1

In [ ]:
plt.figure(figsize=(12, 6))
sns.violinplot(data=donnees_2017, x='state', y='score', inner='quartile')
plt.title(f"Figure {num_graph}: Distribution des scores par région (depuis 2017)")
plt.xlabel("Région")
plt.ylabel("Score")
num_graph+=1

In [ ]:
g = sns.FacetGrid(donnees_2017, col="state", col_wrap=4, height=4, sharey=False)
g.map_dataframe(sns.lineplot, x='annee_mois', y='score', marker='o')
g.set_titles(f"Figure {num_graph}: Région " + "{col_name}")
for ax in g.axes.flatten():
    ax.tick_params(axis='x', rotation=45)
    
num_graph+=1

In [ ]:
df_heat = donnees_2017.groupby(['state', 'annee_mois'])['score'].mean().reset_index()

heat_data = df_heat.pivot(index="state", columns="annee_mois", values="score")

plt.figure(figsize=(12, 8))
sns.heatmap(heat_data, annot=True, fmt=".1f", cmap="coolwarm")
plt.title(f"Figure {num_graph}: Heatmap des scores moyens par mois et par région (depuis 2017)")
plt.xlabel("Année-Mois")
plt.ylabel("Région")
num_graph+=1

In [ ]:
donneesVente['annee_mois'] = donneesVente['purchase_timestamp'].dt.to_period('M').astype(str)

df_grouped = donneesVente.groupby(['annee_mois', 'state']).agg(
    score_moyen=('score', 'mean'),
    nb_commandes=('order_id', 'count')
).reset_index()

df_grouped = df_grouped.sort_values(by='annee_mois')

plt.figure(figsize=(12, 6))

regions = df_grouped['state'].unique()
for region in regions:
    df_region = df_grouped[df_grouped['state'] == region]
    
    plt.plot(df_region['annee_mois'], df_region['score_moyen'], marker='o', label=region)
    
    plt.scatter(df_region['annee_mois'], df_region['score_moyen'], s=df_region['nb_commandes']*2, alpha=0.6)

plt.xticks(rotation=45)
plt.title(f"Figure {num_graph}: Évolution de la moyenne de score par mois par région\n(taille des points proportionnelle au nombre de commandes)")
plt.xlabel("Date (année-mois)")
plt.ylabel("Score moyen")
plt.legend(title="Région")
num_graph+=1

In [ ]:
score_cols = ['score_1', 'score_2', 'score_3', 'score_4', 'score_5']

fig, axs = plt.subplots(2, 3, figsize=(18, 10))
axs = axs.flatten()

for i, col in enumerate(score_cols):
    region_counts = donneesVente.groupby('state')[col].count()
    
    axs[i].pie(region_counts, labels=region_counts.index, autopct='%1.1f%%', startangle=90)
    axs[i].set_title(f"Répartition régionale pour {col}")


axs[-1].axis('off')

plt.suptitle(f"Figure {num_graph}: Camembert de la provenance (région) des notes par score", fontsize=16)
num_graph+=1

In [ ]:
score_cols = ['score_1', 'score_2', 'score_3', 'score_4', 'score_5']
years = [2016, 2017, 2018]

fig, axs = plt.subplots(nrows=5, ncols=3, figsize=(18, 25))

for i, score in enumerate(score_cols):
    for j, year in enumerate(years):
        data_year = donneesVente[donneesVente['purchase_timestamp'].dt.year == year]
        
        region_counts = data_year.groupby('state')[score].count()
        
        axs[i, j].pie(region_counts, labels=region_counts.index, autopct='%1.1f%%', startangle=90)
        axs[i, j].set_title(f"{score} - {year}")

plt.suptitle(f"Figure {num_graph}: Répartition régionale des scores par année", fontsize=16)
num_graph+=1

In [ ]:
donneesVente['month'] = donneesVente['purchase_timestamp'].dt.month

df_count = donneesVente.groupby(['month', 'state'])['score'].count().reset_index(name='note_count')

pivot_count = df_count.pivot(index='state', columns='month', values='note_count')

plt.figure(figsize=(12, 8))
sns.heatmap(pivot_count, annot=True, fmt=".0f", cmap="YlGnBu")
plt.title(f"Figure {num_graph}: Nombre de notes par mois par région")
plt.xlabel("Mois")
plt.ylabel("Région")
num_graph+=1

In [ ]:
comment_cols = ['comment_1', 'comment_2', 'comment_3', 'comment_4', 'comment_5']

donneesVente['avg_comment_length'] = donneesVente[comment_cols].mean(axis=1)

df_chars = donneesVente.groupby(['month', 'state'])['avg_comment_length'].mean().reset_index()

pivot_chars = df_chars.pivot(index='state', columns='month', values='avg_comment_length')

plt.figure(figsize=(12, 8))
sns.heatmap(pivot_chars, annot=True, fmt=".1f", cmap="YlOrRd")
plt.title(f"Figure {num_graph}: Longueur moyenne des commentaires par mois par région")
plt.xlabel("Mois")
plt.ylabel("Région")
num_graph+=1

In [ ]:
score_columns = ['score_1', 'score_2', 'score_3', 'score_4', 'score_5']
comment_columns = ['comment_1', 'comment_2', 'comment_3', 'comment_4', 'comment_5']

all_scores = []
all_comments = []

for i in range(5):
    all_scores.extend(donneesVente[score_columns[i]].dropna().tolist())
    all_comments.extend(donneesVente[comment_columns[i]].dropna().tolist())

df_combined = pd.DataFrame({'score': all_scores, 'comment_length': all_comments})

df_combined = df_combined[df_combined['score'].between(1, 5)]

grouped_df = df_combined.groupby('score')['comment_length'].mean().reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(x='score', y='comment_length', data=grouped_df, marker='o')

plt.title("Relation entre la longueur moyenne des commentaires et les scores")
plt.xlabel("Score attribué")
plt.ylabel("Longueur moyenne du commentaire (nombre de caractères)")

## Achat

In [ ]:
plt.figure(figsize=(14, 8))
sns.histplot(data=donneesAchat, x='price', hue='cust_state', bins=30, kde=True, alpha=0.6)


plt.title(f"Figure {num_graph}: Histogramme du revenu par région", fontsize=16)
plt.xlabel('Revenu', fontsize=14)
plt.ylabel('Fréquence d\'achat', fontsize=14)
plt.legend(title='Région', fontsize=10, bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, linestyle='--', alpha=0.6)
num_graph+=1

In [ ]:
plt.figure(figsize=(14, 8))
sns.boxplot(
    data=donneesAchat,
    x='cust_state', 
    y='price',
    palette='Set3'
)


plt.title(f"Figure {num_graph}: Box Plot de la distribution de revenu par région", fontsize=16)
plt.xlabel('Région', fontsize=14)
plt.ylabel('Revenue', fontsize=14)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.6)
num_graph+=1

In [ ]:
donneesAchat['rendement'] = donneesAchat['price'] - donneesAchat['freight_value']
rendement_moyen = donneesAchat.groupby(['cust_state', 'annee'])['rendement'].mean().unstack()


plt.figure(figsize=(12, 8))
sns.heatmap(rendement_moyen, cmap='YlGnBu', annot=True, fmt=".1f", linewidths=0.5, cbar_kws={'label': 'Moyenne de rendement'})


plt.title(f"Figure {num_graph}: Heatmap du rendement moyen par région par année", fontsize=16)
plt.xlabel('Année', fontsize=14)
plt.ylabel('Région', fontsize=14)
plt.xticks(rotation=45)
num_graph+=1

In [ ]:
yearly_trends = donneesAchat.groupby(['cust_state', 'annee']).agg({
    'price': 'mean',
    'freight_value': 'mean'
}).reset_index()

g = sns.FacetGrid(yearly_trends, col='cust_state', col_wrap=5, height=4, sharey=False)
g.map(sns.lineplot, 'annee', 'price', label='Avg Revenue', color='blue')
g.map(sns.lineplot, 'annee', 'freight_value', label='Avg Freight', color='orange')

g.set_titles('{col_name}')
g.set_axis_labels('Année', 'Valeurs')
g.add_legend(title='Metrics')

plt.subplots_adjust(top=0.9)
g.fig.suptitle(f"Figure {num_graph}: Tendance annuel du revenu et des coûts de transports par régions", fontsize=16)
num_graph+=1

In [ ]:
donneesAchat['profitability'] = donneesAchat['price'] - donneesAchat['freight_value']
profit_mensuel = donneesAchat.groupby(['cust_state', 'mois'])['profitability'].mean().unstack()


plt.figure(figsize=(14, 8))
sns.heatmap(
    profit_mensuel, 
    cmap='coolwarm', 
    annot=True, 
    fmt=".1f", 
    linewidths=0.5, 
    cbar_kws={'label': 'Average Profitability'}
)


plt.title(f"Figure {num_graph}: Heatmap de la profibabilité des régions par mois", fontsize=16)
plt.xlabel('Mois', fontsize=14)
plt.ylabel('Régions', fontsize=14)
num_graph+=1

In [ ]:
valeurs_moyennes = donneesAchat.groupby(['cust_state', 'annee'])[['freight_value', 'price']].mean().reset_index()

plt.figure(figsize=(16, 10))

sns.barplot(data=valeurs_moyennes, x='annee', y='freight_value', hue='cust_state', ci=None, palette='Blues')

sns.barplot(data=valeurs_moyennes, x='annee', y='price', hue='cust_state', ci=None, palette='Reds', alpha=0.6)

plt.title(f"Figure {num_graph}: Coût transport moyen et prix moyen par région par année", fontsize=16)
plt.xlabel('Année', fontsize=14)
plt.ylabel('Valeur moyenne', fontsize=14)
plt.legend(title='State', bbox_to_anchor=(1.05, 1), loc='upper left')
num_graph+=1

In [ ]:
valeurs_moyennes = donneesAchat.groupby(['cust_state', 'mois'])[['freight_value', 'price']].mean().reset_index()

plt.figure(figsize=(16, 10))

sns.barplot(data=valeurs_moyennes, x='mois', y='freight_value', hue='cust_state', ci=None, palette='Blues')

sns.barplot(data=valeurs_moyennes, x='mois', y='price', hue='cust_state', ci=None, palette='Reds', alpha=0.6)

plt.title(f"Figure {num_graph}: Coût moyen du transport et prix moyen par région par mois", fontsize=16)
plt.xlabel('Mois', fontsize=14)
plt.ylabel('Valeur moyenne', fontsize=14)
plt.legend(title='State', bbox_to_anchor=(1.05, 1), loc='upper left')
num_graph+=1

In [ ]:
donneesAchat_filtered = donneesAchat[donneesAchat['annee'] >= 2017]


evolution_prix_categorie = donneesAchat_filtered.groupby(['category_name', 'annee_mois'])['price'].mean().reset_index()


plt.figure(figsize=(16, 10))


sns.lineplot(data=evolution_prix_categorie, x='annee_mois', y='price', hue='category_name', marker='o')


plt.title(f"Figure {num_graph}: Evolution du prix moyen pour chaque categorie de produit", fontsize=16)
plt.xlabel('Année-Mois', fontsize=14)
plt.ylabel('Prix moyen', fontsize=14)
plt.xticks(rotation=45)
plt.legend(title='Categorie de produit', bbox_to_anchor=(1.05, 1), loc='upper left')
num_graph+=1

In [ ]:
donneesAchat_filtered = donneesAchat[donneesAchat['annee'] >= 2017]

evolution_commande_temps = donneesAchat_filtered.groupby(['category_name', 'annee_mois'])['order_id'].nunique().reset_index()

plt.figure(figsize=(16, 10))

sns.lineplot(data=evolution_commande_temps, x='annee_mois', y='order_id', hue='category_name', marker='o')

plt.title(f"Figure {num_graph}: Evolution des commandes de chaque catégorie de produit", fontsize=16)
plt.xlabel('Année-Mois', fontsize=14)
plt.ylabel('Nombre de commande', fontsize=14)
plt.xticks(rotation=45)
plt.legend(title='Catégorie de produit', bbox_to_anchor=(1.05, 1), loc='upper left')
num_graph+=1

In [ ]:
state_data = donneesAchat.groupby('cust_state').agg({
    'price': 'mean',
    'freight_value': 'mean',
    'order_id': 'count'
}).reset_index()


state_data.columns = ['Région', 'Prix moyen', 'Moyenne du prix du transport', 'Nombre de commande']


plt.figure(figsize=(14, 8))
bubble = plt.scatter(
    state_data['Prix moyen'], state_data['Moyenne du prix du transport'], 
    s=state_data['Nombre de commande'] * 10,
    c=state_data['Nombre de commande'], cmap='viridis', alpha=0.7, edgecolor='k'
)

for i, row in state_data.iterrows():
    plt.text(row['Prix moyen'], row['Moyenne du prix du transport'], row['Région'], fontsize=9, ha='center')


plt.title(f"Figure {num_graph}: Potentiel d\'investissement par région", fontsize=16)
plt.xlabel('Revenu moyen', fontsize=14)
plt.ylabel('Moyenne du prix du transport', fontsize=14)
plt.colorbar(bubble, label='Nombre de commande')
plt.grid(True, linestyle='--', alpha=0.6)
num_graph+=1

### Analyse 2

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi/2.0)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda/2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

In [ ]:
plt.figure(figsize=(12, 8))
sns.scatterplot(
    x='cust_lng', y='cust_lat', data=donneesAchat, 
    color='blue', alpha=0.5, label='Customers'
)
sns.scatterplot(
    x='sell_lng', y='sell_lat', data=donneesAchat, 
    color='red', alpha=0.5, label='Sellers'
)


plt.title(f"Figure {num_graph}: Seller and Customer Positions", fontsize=16)
plt.xlabel('Longitude', fontsize=14)
plt.ylabel('Latitude', fontsize=14)
plt.legend(loc='upper right', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
num_graph+=1

In [ ]:
donneesAchat['delivery_time'] = (donneesAchat['delivered_customer'] - donneesAchat['estimated_delivery']).dt.days

donneesAchat = donneesAchat.dropna(subset=['delivery_time'])

delivery_performance = donneesAchat.groupby('cust_state')['delivery_time'].mean().reset_index()

plt.figure(figsize=(12, 8))
sns.boxplot(data=donneesAchat, x='cust_state', y='delivery_time')

plt.xticks(rotation=90)
plt.title(f"Figure {num_graph}: Delivery Time Performance by State")
plt.xlabel('Customer State')
plt.ylabel('Delivery Time (days)')
num_graph+=1

In [ ]:
customer_counts = donneesAchat.groupby('cust_state')['customer_unique_id'].nunique()
seller_counts = donneesAchat.groupby('sell_state')['seller_id'].nunique()

state_counts = pd.DataFrame({
    'Customers': customer_counts,
    'Sellers': seller_counts
}).fillna(0)

plt.figure(figsize=(14, 8))
state_counts.plot(kind='bar', figsize=(14, 8), alpha=0.8, width=0.7)

plt.title(f"Figure {num_graph}: Number of Customers and Sellers by State", fontsize=16)
plt.xlabel('State', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.xticks(rotation=45)
plt.legend(title='Category', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
num_graph+=1

In [ ]:
donneesAchat['distance_km'] = haversine(donneesAchat['sell_lat'], donneesAchat['sell_lng'],
                                        donneesAchat['cust_lat'], donneesAchat['cust_lng'])

donneesAchat['time_delivery_hrs'] = (donneesAchat['delivered_customer'] - donneesAchat['approved_at']).dt.total_seconds() / 3600

plt.figure(figsize=(12,6))
sns.scatterplot(data=donneesAchat, x='distance_km', y='time_delivery_hrs', alpha=0.5)
plt.title(f"Figure {num_graph}: Distance vs Temps de livraison")
plt.xlabel("Distance (km)")
plt.ylabel("Temps de livraison (heures)")
num_graph+=1

In [ ]:
df_orders = donneesAchat.groupby(['seller_id', 'annee_mois'])['order_id'].nunique().reset_index(name='order_count')

top_sellers = df_orders.groupby('seller_id')['order_count'].sum().nlargest(5).index
df_top = df_orders[df_orders['seller_id'].isin(top_sellers)]

plt.figure(figsize=(14, 8))
sns.lineplot(data=df_top, x='annee_mois', y='order_count', hue='seller_id', marker='o')
plt.title(f"Figure {num_graph}: Evolution du nombre de commandes par vendeur (Top 5) à travers le temps")
plt.xlabel("Mois")
plt.ylabel("Nombre de commandes")
plt.xticks(rotation=45)
plt.legend(title="Vendeur")
num_graph+=1

In [ ]:
df_heat = donneesAchat.groupby(['cust_state', 'mois'])['time_delivery_hrs'].mean().reset_index()

pivot_heat = df_heat.pivot(index='cust_state', columns='mois', values='time_delivery_hrs')

plt.figure(figsize=(12, 8))
sns.heatmap(pivot_heat, annot=True, fmt=".1f", cmap="YlOrRd")
plt.title(f"Figure {num_graph}: Temps de livraison moyen (heures) par région selon les mois")
plt.xlabel("Mois")
plt.ylabel("Région (cust_state)")
num_graph+=1

In [ ]:
donneesAchat['approved_at'] = pd.to_datetime(donneesAchat['approved_at'])
donneesAchat['delivered_customer'] = pd.to_datetime(donneesAchat['delivered_customer'])
donneesAchat['purchase_timestamp'] = pd.to_datetime(donneesAchat['purchase_timestamp'])

donneesAchat2017 = donneesAchat[donneesAchat['annee'] >= 2017]

donneesAchat2017['time_delivery_hrs'] = (
    donneesAchat2017['delivered_customer'] - donneesAchat2017['approved_at']
).dt.total_seconds() / 3600

df_graph1 = donneesAchat2017.groupby('annee_mois')['time_delivery_hrs'].mean().reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(data=df_graph1, x='annee_mois', y='time_delivery_hrs', marker='o')
plt.title(f"Figure {num_graph}: Évolution du temps de livraison moyen par mois")
plt.xlabel("Année-Mois")
plt.ylabel("Temps de livraison moyen (heures)")
plt.xticks(rotation=45)
num_graph+=1

In [ ]:
df_graph2 = donneesAchat2017.groupby(['annee_mois', 'cust_state'])['time_delivery_hrs'].mean().reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(data=df_graph2, x='annee_mois', y='time_delivery_hrs', hue='cust_state', marker='o')
plt.title(f"Figure {num_graph}: Évolution du temps de livraison moyen par mois selon la région")
plt.xlabel("Année-Mois")
plt.ylabel("Temps de livraison moyen (heures)")
plt.xticks(rotation=45)
plt.legend(title="Région", bbox_to_anchor=(1.05, 1), loc='upper left')
num_graph+=1

In [ ]:
donneesAchat2017['distance_km'] = haversine(
    donneesAchat['sell_lat'], donneesAchat['sell_lng'],
    donneesAchat['cust_lat'], donneesAchat['cust_lng']
)

df_graph5 = donneesAchat2017.groupby('annee_mois')['distance_km'].mean().reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(data=df_graph5, x='annee_mois', y='distance_km', marker='o')
plt.title(f"Figure {num_graph}: Évolution de la distance moyenne entre vendeur et client par mois")
plt.xlabel("Année-Mois")
plt.ylabel("Distance moyenne (km)")
plt.xticks(rotation=45)
num_graph+=1

## Analyse 3

In [ ]:
df_orders = donneesAchat.drop_duplicates(subset=['order_id'])

In [ ]:
orders_per_month = df_orders.groupby('annee_mois').size()

In [ ]:
plt.figure(figsize=(12, 6))
orders_per_month.plot(kind='line', marker='o', linestyle='-', color='blue')
plt.title(f"Figure {num_graph}: Évolution temporelle des commandes")
plt.xlabel("Année-Mois")
plt.ylabel("Nombre de commandes")
plt.xticks(rotation=45)
num_graph+=1

In [ ]:
top20_categories = donneesAchat['category_name'].value_counts().head(20)

plt.figure(figsize=(10, 6))
sns.barplot(x=top20_categories.values, y=top20_categories.index, palette="viridis")
plt.title(f"Figure {num_graph}: Top 20 catégories avec le plus de ventes")
plt.xlabel("Nombre de commandes")
plt.ylabel("Catégorie")
num_graph+=1

In [ ]:
day_mapping = {1: 'Lundi', 2: 'Mardi', 3: 'Mercredi', 4: 'Jeudi',
               5: 'Vendredi', 6: 'Samedi', 7: 'Dimanche'}

donneesAchat['jour_semaine_nom'] = donneesAchat['jour_semaine'].map(day_mapping)

pivot_heatmap = donneesAchat.pivot_table(index='jour_semaine_nom', columns='heure', 
                               values='order_id', aggfunc='count')

ordre_jours = ['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche']
pivot_heatmap = pivot_heatmap.reindex(ordre_jours)

plt.figure(figsize=(12, 6))
sns.heatmap(pivot_heatmap, annot=True, fmt=".0f", cmap="YlGnBu")
plt.title(f"Figure {num_graph}: Volume des commandes par jour de semaine et heure")
plt.xlabel("Heure")
plt.ylabel("Jour de la semaine")
num_graph+=1

In [ ]:
df_returns = donneesAchat[donneesAchat['status'] == 'canceled']

day_mapping = {1: 'Lundi', 2: 'Mardi', 3: 'Mercredi', 4: 'Jeudi', 5: 'Vendredi', 6: 'Samedi', 7: 'Dimanche'}
df_returns['jour_semaine_nom'] = df_returns['jour_semaine'].map(day_mapping)

pivot_returns = df_returns.pivot_table(index='jour_semaine_nom', columns='heure', values='order_id', aggfunc='count')

ordre_jours = ['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche']
pivot_returns = pivot_returns.reindex(ordre_jours)

plt.figure(figsize=(12, 6))
sns.heatmap(pivot_returns, annot=True, fmt=".0f", cmap="YlOrRd")
plt.title(f"Figure {num_graph}: Activité horaire des retours/reclamations (statut 'canceled')")
plt.xlabel("Heure")
plt.ylabel("Jour de la semaine")
num_graph+=1

In [ ]:
pivot_heatmap = donneesAchat.pivot_table(index='mois', columns='heure', values='order_id', aggfunc='count')

pivot_heatmap = pivot_heatmap.sort_index()

plt.figure(figsize=(12, 6))
sns.heatmap(pivot_heatmap, annot=True, fmt=".0f", cmap="YlGnBu")
plt.title(f"Figure {num_graph}: Volume des commandes par mois et heure")
plt.xlabel("Heure")
plt.ylabel("Mois")
num_graph+=1

In [ ]:
donneesAchat['mois'] = donneesAchat['purchase_timestamp'].dt.month
donneesAchat['jour_semaine'] = donneesAchat['purchase_timestamp'].dt.dayofweek

jours_semaine = {0: "Lundi", 1: "Mardi", 2: "Mercredi", 3: "Jeudi", 4: "Vendredi", 5: "Samedi", 6: "Dimanche"}
donneesAchat['jour_semaine'] = donneesAchat['jour_semaine'].map(jours_semaine)

df_grouped = donneesAchat.groupby(['mois', 'jour_semaine'])['order_id'].count().reset_index()

plt.figure(figsize=(12, 6))
sns.barplot(data=df_grouped, x="jour_semaine", y="order_id", hue="mois", palette="viridis")
plt.xlabel("Jour de la semaine")
plt.ylabel("Nombre de commandes")
plt.title(f"Figure {num_graph}: Nombre de commandes par jour de la semaine et par mois")
plt.xticks(rotation=45)
plt.legend(title="Mois")
num_graph+=1

In [ ]:
df_client_orders = donneesAchat.groupby('customer_unique_id')['order_id'].nunique().reset_index()
df_client_orders['type_client'] = df_client_orders['order_id'].apply(lambda x: 'Nouveau' if x == 1 else 'Récurrent')

df_panier_moyen = donneesAchat.merge(df_client_orders[['customer_unique_id', 'type_client']], on='customer_unique_id')
df_panier_moyen = df_panier_moyen.groupby('type_client')['price'].mean().reset_index()

plt.figure(figsize=(8, 5))
sns.barplot(data=df_panier_moyen, x='type_client', y='price', palette=["#FF9999", "#66B3FF"])
plt.xlabel("Type de client")
plt.ylabel("Panier moyen (€)")
plt.title(f"Figure {num_graph}: Panier moyen par type de client")
num_graph+=1

In [ ]:
df_filtered = donneesAchat[donneesAchat['purchase_timestamp'].dt.year >= 2017]

df_filtered['annee_mois'] = df_filtered['purchase_timestamp'].dt.to_period('M')
df_panier_moyen = df_filtered.groupby('annee_mois')['price'].mean().reset_index()

df_panier_moyen['annee_mois'] = df_panier_moyen['annee_mois'].astype(str)

plt.figure(figsize=(12, 6))
plt.plot(df_panier_moyen['annee_mois'], df_panier_moyen['price'], marker='o', linestyle='-', color='b', label='Panier Moyen')
plt.xticks(rotation=45, ha='right')
plt.xlabel("Temps (Année-Mois)")
plt.ylabel("Panier Moyen (€)")
plt.title(f"Figure {num_graph}: Évolution du panier moyen à travers le temps (à partir de 2017)")
plt.legend()
num_graph+=1

In [ ]:
df_interactions = donneesAchat.groupby(['customer_unique_id', 'seller_id'])['order_id'].count().reset_index()
df_interactions.columns = ['customer_unique_id', 'seller_id', 'nombre_commandes']

df_prob = df_interactions.groupby('customer_unique_id')['nombre_commandes'].max().reset_index()
df_prob['recommande'] = df_prob['nombre_commandes'].apply(lambda x: 'Oui' if x > 1 else 'Non')

proportions = df_prob['recommande'].value_counts(normalize=True) * 100

plt.figure(figsize=(6, 6))
plt.pie(proportions, labels=proportions.index, autopct='%1.1f%%', colors=["#66B3FF", "#FF9999"], startangle=90)
plt.title(f"Figure {num_graph}: Probabilité qu'un client commande chez le même vendeur")
num_graph+=1

In [ ]:
end_time = time.time()

execution_time = end_time - start_time

print(f"Total execution time: {execution_time:.2f} seconds")